In [2]:
# https://gist.github.com/arngarden/6087798
import theano
from pylearn2.models import mlp
from pylearn2.training_algorithms import sgd
from pylearn2.termination_criteria import EpochCounter
from pylearn2.datasets.dense_design_matrix import DenseDesignMatrix
import numpy as np
from random import randint

In [3]:
class XOR(DenseDesignMatrix):
    def __init__(self):
        self.class_names = ['0', '1']
        X = [[randint(0, 1), randint(0, 1)] for _ in range(1000)]
        y = []
        for a, b in X:
            if a + b == 1:
                y.append([0, 1])
            else:
                y.append([1, 0])
        X = np.array(X)
        y = np.array(y)
        super(XOR, self).__init__(X=X, y=y)

In [4]:
# create XOR dataset
ds = XOR()

In [5]:
print ds.X.shape
print ds.y.shape

(1000, 2)
(1000, 2)


In [6]:
print ds.X[0]
print ds.y[0]

[1 0]
[0 1]


In [5]:
# create hidden layer with 2 nodes, init weights in range -0.1 to 0.1 and add
# a bias with value 1
hidden_layer = mlp.Sigmoid(layer_name='hidden', dim=2, irange=.1, init_bias=1.)

In [6]:
# create Softmax output layer
output_layer = mlp.Softmax(2, 'output', irange=.1)

In [7]:
# create Stochastic Gradient Descent trainer that runs for 400 epochs
trainer = sgd.SGD(learning_rate=.05, batch_size=10, termination_criterion=EpochCounter(400))

In [8]:
layers = [hidden_layer, output_layer]

In [9]:
# create neural net that takes two inputs
ann = mlp.MLP(layers, nvis=2)
trainer.setup(ann, ds)

Parameter and initial learning rate summary:
	hidden_W: 0.05
	hidden_b: 0.05
	softmax_b: 0.05
	softmax_W: 0.05
Compiling sgd_update...
Compiling sgd_update done. Time elapsed: 1.533082 seconds


In [10]:
# train neural net until the termination criterion is true
while True:
    trainer.train(dataset=ds)
    ann.monitor.report_epoch()
    ann.monitor()
    if not trainer.continue_learning(ann):
        break

compiling begin_record_entry...
compiling begin_record_entry done. Time elapsed: 0.013912 seconds
Monitored channels: 
Compiling accum...
Compiling accum done. Time elapsed: 0.000534 seconds
Monitoring step:
	Epochs seen: 1
	Batches seen: 100
	Examples seen: 1000
Monitoring step:
	Epochs seen: 2
	Batches seen: 200
	Examples seen: 2000
Monitoring step:
	Epochs seen: 3
	Batches seen: 300
	Examples seen: 3000
Monitoring step:
	Epochs seen: 4
	Batches seen: 400
	Examples seen: 4000
Monitoring step:
	Epochs seen: 5
	Batches seen: 500
	Examples seen: 5000
Monitoring step:
	Epochs seen: 6
	Batches seen: 600
	Examples seen: 6000
Monitoring step:
	Epochs seen: 7
	Batches seen: 700
	Examples seen: 7000
Monitoring step:
	Epochs seen: 8
	Batches seen: 800
	Examples seen: 8000
Monitoring step:
	Epochs seen: 9
	Batches seen: 900
	Examples seen: 9000
Monitoring step:
	Epochs seen: 10
	Batches seen: 1000
	Examples seen: 10000
Monitoring step:
	Epochs seen: 11
	Batches seen: 1100
	Examples seen: 11000


In [11]:
inputs = np.array([[0, 0]])
print ann.fprop(theano.shared(inputs, name='inputs')).eval()
inputs = np.array([[0, 1]])
print ann.fprop(theano.shared(inputs, name='inputs')).eval()
inputs = np.array([[1, 0]])
print ann.fprop(theano.shared(inputs, name='inputs')).eval()
inputs = np.array([[1, 1]])
print ann.fprop(theano.shared(inputs, name='inputs')).eval()

[[ 0.96310425  0.03689575]]
[[ 0.04224482  0.95775518]]
[[ 0.04463144  0.95536856]]
[[ 0.97087054  0.02912946]]


In [12]:
ann.fprop

<bound method MLP.fprop of <pylearn2.models.mlp.MLP object at 0x1042d3610>>